<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
#Keras provides the img_to_array() function for converting a loaded image in PIL format
#into a NumPy array for use with deep learning models
from tensorflow.keras.preprocessing.image import img_to_array
#array_to_img() function that can be used for converting a NumPy array of pixel data into a PIL
#Converts a class vector (integers) to binary class matrix.
from tensorflow.keras.utils import to_categorical
#To arrange the Keras layers in a sequential order
from tensorflow.keras.models import Sequential
#Flatten layer is used to make the multidimensional input one-dimensional,
#commonly used in the transition from the convolution layer to the full connected layer.
#defines the size of the output from the dense layer
from tensorflow.keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/gender_dataset_face.zip'

Archive:  /content/drive/MyDrive/gender_dataset_face.zip
  inflating: gender_dataset_face/woman/face_180.jpg  
  inflating: gender_dataset_face/woman/face_648.jpg  
  inflating: gender_dataset_face/woman/face_133.jpg  
  inflating: gender_dataset_face/woman/face_296.jpg  
  inflating: gender_dataset_face/woman/face_647.jpg  
  inflating: gender_dataset_face/woman/face_327.jpg  
  inflating: gender_dataset_face/woman/face_1047.jpg  
  inflating: gender_dataset_face/woman/face_171.jpg  
  inflating: gender_dataset_face/woman/face_635.jpg  
  inflating: gender_dataset_face/woman/face_1082.jpg  
  inflating: gender_dataset_face/woman/face_1274.jpg  
  inflating: gender_dataset_face/woman/face_1233.jpg  
  inflating: gender_dataset_face/woman/face_1211.jpg  
  inflating: gender_dataset_face/woman/face_896.jpg  
  inflating: gender_dataset_face/woman/face_855.jpg  
  inflating: gender_dataset_face/woman/face_288.jpg  
  inflating: gender_dataset_face/woman/face_1269.jpg  
  inflating: gender

# save all images into image_files var

In [ ]:
image_files = [f for f in glob.glob(r'/content/gender_dataset_face/**/*.jpg', recursive=True) if not os.path.isdir(f)]
image_files

The code snippet you provided is a Python code that does the following:

1. **Imports Required Libraries**: It assumes that you have `glob` and `os` libraries already imported. `glob` is used for pathname searching, and `os` is used for interacting with the operating system.

2. **Search for Image Files**:
   - `glob.glob(r'/content/gender_dataset_face/**/*.jpg', recursive=True)` searches for all files with the `.jpg` extension within the directory `/content/gender_dataset_face/` and its subdirectories (due to `**`).
   - The `recursive=True` argument indicates that it should include files from all levels of subdirectories.

3. **Filter Out Directories**:
   - The code then filters this list to exclude any directories. The condition `if not os.path.isdir(f)` checks if the found item is not a directory.

4. **Create a List of Image File Paths**:
   - The result is stored in the list `image_files`, which contains the paths of all `.jpg` files found in the specified directory and its subdirectories, excluding any directories.

5. **Return the List**: Finally, displaying `image_files` provides the list of image file paths.

In summary, this code finds and lists all JPEG image files in a specified folder and its subfolders, while ignoring any directories.


In [ ]:
image_files[-1]

In [ ]:
random.shuffle(image_files)

In [ ]:
image_dimension = (100, 100, 3)

# creating an empty list

In [ ]:
data = []
labels = []

In [ ]:
img = '/content/gender_dataset_face/woman/face_0.jpg'

In [ ]:
label = img.split(os.path.sep)
print(label[-2])

In [ ]:
print(label[-2])

In [ ]:
for img in image_files:
    image = cv2.imread(img)
    image = cv2.resize(image, (image_dimension[0], image_dimension[1]))
    image = img_to_array(image)
    data.append(image)
    label = img.split(os.path.sep)[-2]
    if label=="woman":
        label=1
    else:
        label=0
    labels.append([label])

In [ ]:
print(labels)

# Now we will convert this into a Numpy Array

In [ ]:
data = np.array(data, dtype = 'float')

In [ ]:
labels = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train[:1])

In [ ]:
print(y_train[1])

In [ ]:
print(y_train[-1])

# Expanding the dimensions

In [ ]:
trainy = to_categorical(y_train, num_classes = 2)
testy = to_categorical(y_test, num_classes=2)

In [ ]:
trainy[1]

In [ ]:
trainy[-1]

In [ ]:
augmentation_data = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'  # Specifies the strategy to fill in newly created pixels.
)

In [ ]:
Vgg16 = VGG16(input_shape=(100,100,3), weights='imagenet', include_top=False) # Defining the VGG16 model

for layers in Vgg16.layers: # Iterating over layers in Vgg16 (note the capital 'V')
    layers.trainable = False # Setting each layer to non-trainable

In [ ]:
x = Flatten()(Vgg16.output) # Changed 'vgg16' to 'Vgg16'
folders = glob.glob(r'/content/gender_dataset_face/*')
len(folders)

In [ ]:
prediction = Dense (len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=Vgg16.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 32

In [ ]:
history = model.fit(
    augmentation_data.flow(X_train, trainy, batch_size=batch_size),
    validation_data=(X_test, testy),
    epochs=10
)